# robot image table

> watches all of the images from the robot tests, grouped by folder

- Restart and run all, then click the _Run 🤖_ button

In [ ]:
from pathlib import Path
from base64 import b64encode
import asyncio
import subprocess
import ipywidgets as W
import IPython, pandas as pd
import time
from tornado.ioloop import IOLoop
HERE = Path().parent.resolve()
ROOT = HERE.parent.resolve()
AOUT = ROOT / "atest" / "output"

In [ ]:
out = W.HTML()
status = W.HTML(value="Click the button!")
btn = W.Button(description="🤖", button_style="primary", icon="play", layout=dict(width="5em"))
btn.on_click(lambda *args: IOLoop.current().add_callback(run))
W.VBox([W.HBox([btn, status]), out])

In [ ]:
%%html
<style>
table {width: 100%; border: 0}
th, td { vertical-align: top; border: 0}
td div{display:flex; flex-direction: column; justify-content: top; align-items: center}
td div img {max-width: 100%; margin-bottom: 4px;}
</style>

In [ ]:
def quilt():
    imgs = sorted([p for p in AOUT.rglob("*.png") if "pabot_results" in str(p)])
    df = pd.DataFrame(imgs, columns=["path"])
    df["folder"] = df["path"].apply(lambda x: str(x.parent.relative_to(AOUT)).split("screens")[-1])
    df["name"] = df["path"].apply(lambda x: x.name)
    df["modified"] = df["path"].apply(lambda x: x.stat().st_mtime)
    df = df.sort_values(["folder", "modified"])
    df = df.set_index(["folder"])
    df["img"] = df["path"].apply(lambda x: f"""{x.name} <img src="data:image/png;base64,{b64encode(x.read_bytes()).decode("utf-8")}"/>""")
    by_row = pd.DataFrame(df.groupby(['folder'])['img'].transform(lambda x: f"""<div>{" ".join(x[::-1])}</div>"""))
    return by_row.reset_index().set_index(["folder"]).drop_duplicates()

In [ ]:
def update(start):
    try:
        df = quilt().T
        out.value = "\n".join([
            df.to_html(escape=False)
        ])
        if len(df):
            status.value = f"{df.size} images in {int(time.time() - start)}s"
            return
    except:
        pass

In [ ]:
async def run():
    btn.button_style = "warning"
    out.value = "waiting to start..."
    status.value = "forgetting..."
    subprocess.call(["doit", "forget", "robot"], cwd=ROOT)
    status.value = "starting..."
    proc = subprocess.Popen(["doit", "robot"], cwd=ROOT)
    start = time.time()
    proc.poll()
    while proc.returncode is None:
        await asyncio.sleep(1)
        proc.poll()
        update(start)
    
    status.value += f" <strong>done</strong> <code>{proc.returncode}</code>"
    
    for log in AOUT.rglob("log.html"):
        status.value += f""" <a href="{log.as_uri()}" target="_blank">{log.parent.name}</a>"""
    
    if proc.returncode == 0:
        btn.button_style = "success"
    else:
        btn.button_style = "danger"